# Range Accrual Callable Range Accrual with Fixed Funding Leg

This notebook demonstrates how to access and use the functionalities of **Callable Range Accrual with fixed funding leg** within LSEG Financial Analytics SDK.

**You will be able to:**
* Define Range Accrual instrument with a fixed funding leg and callable
* Define call schedule 
* Configure pricing parameters
* Evaluate Analytics (Description, Price and Sensitivities)

A range accrual swap is a derivative contract that allows two parties to exchange cash flows based on the performance of an underlying asset within a specified range, typically used to manage interest rate risk.

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
import lseg_analytics.pricing.instruments.structured_products as sp

import datetime as dt
import pandas as pd
import json
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Range Accrual, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (underlying rate, payoff description and schedule, dates, notional)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing models and parameters, optional

In [2]:
# 1. Create SP definition object

print("Step 1: Configuring instrument definition...")

callable_RA_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Callable_RangeAccrual",
    inputs = [
        sp.NameTypeValue(name="StartDate", type = "date", value=dt.date(2025, 9, 15)),
        sp.NameTypeValue(name="EndDate", type = "date", value= dt.date(2030, 9, 15)),
        sp.NameTypeValue(name="Underlying", type = "string", value="EUR"),
        sp.NameTypeValue(name="Notional", type = "string", value="1000000"),
        sp.NameTypeValue(name="PayOrReceiveRA", type = "string", value="Receive"),
        sp.NameTypeValue(name="CMSTenor", type = "string", value="10Y"),
        sp.NameTypeValue(name="CMSRate", type = "string", value="Cmsrate(Underlying,PeriodStart(),CMSTenor)"),
        sp.NameTypeValue(name="UpperBound", type = "string", value="3.2%"),
        sp.NameTypeValue(name="LowerBound", type = "string", value="1.15%"),
        sp.NameTypeValue(name="Coupon", type = "string", value="3.9%"),
        sp.NameTypeValue(name="Frequency", type = "string", value="SemiAnnual"),
		sp.NameTypeValue(name="ObservationFrequency", type = "string", value="Monthly"),
        sp.NameTypeValue(name="DayCount", type = "string", value="30/360"),
        sp.NameTypeValue(name="PayOrReceiveFixed", type = "string", value="Pay"),
        sp.NameTypeValue(name="FixedFrequency", type = "string", value="SemiAnnual"),
        sp.NameTypeValue(name="FixedDayCount", type = "string", value="30/360"),
        sp.NameTypeValue(name="FixedCoupon", type = "string", value="0.9%"),
        sp.NameTypeValue(name="FirstCallDate", type = "date", value=dt.date(2025, 9, 15)),
		sp.NameTypeValue(name="CallFrequency", type = "string", value="SemiAnnual"),
        sp.NameTypeValue(name="CallNoticeGap", type = "string", value="-5b"),
        sp.NameTypeValue(name="CallOwner", type = "string", value="Counterparty"),
        
    ],
    payoff_description = [
			[
				"Schedule type",
				"Schedule description",
				"RangeAccrualLeg",
				"FixedRateLeg",
                "Reinitialisation",
				"Price",
				"PricePercent"
			],
			[
				"AllTheTime",
				"FromTo(DateTable(StartDate,EndDate,Frequency),ObservationFrequency)",
				"$n1 = if(abs(CMSRate)>LowerBound and CMSRate<UpperBound, $n1+1, $n1); $n2 =$n2+1",
				"",
				"",
				"",
                ""
			],
            [
				"OnSchedule PeriodEnd",
				"DateTable(StartDate, EndDate, Frequency, Daycount)",
				"PayOrReceiveRA Coupon*$n1/$n2*InterestTerm()*Notional",
				"",
				"$n1 = 0; $n2 = 0",
				"PayOrReceiveRA Coupon*$n1/$n2*InterestTerm()*Notional",
                "PayOrReceiveRA Coupon*$n1/$n2*InterestTerm()*100"
			],
            [
				"OnSchedule PeriodEnd",
				"DateTable(StartDate, EndDate, FixedFrequency, FixedDaycount)",
				"",
				"PayOrReceiveFixed FixedCoupon*InterestTerm()*Notional",
				"",
				"PayOrReceiveFixed FixedCoupon*InterestTerm()*Notional",
                "PayOrReceiveFixed FixedCoupon*InterestTerm()*100"
			],
            [
				"OnSchedule",
				"DateTable(FirstCallDate, EndDate, CallFrequency,  ResetGap:= CallNoticeGap, Arrear:=Yes)",
				"",
				"",
				"",
				"CallableBy(CallOwner,0)",
                "CallableBy(CallOwner,0)/Notional*100"
			],
            [
                "AtDate",
                "AsOfDate()",
                "",
                "",
                "",
                "",
                "Report(ColumnVal(Price)/Notional*100)"
            ]
		]
)

print("	Instrument definition configured")

# 2. Create SP instrument definition object
print("Step 2: Creating instrument definition object...")

callable_RA = sp.StructuredProductsDefinitionInstrument(definition = callable_RA_definition)
print("	Instrument definition created")


# 3. Create SP parameters object - optional
print("Step 3: Configuring pricing parameters...")

callable_RA_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 9, 12),  # Set your desired valuation date
    numerical_method = sp.GenericNumericalMethod(method="AmericanMonteCarlo"),
    models=[sp.ModelDefinition(
            underlying_code = "EUR",
            underlying_tag = "EUR",
            underlying_currency = "EUR",
            asset_class = "InterestRate",
            model_name= "HullWhite1Factor",
            calibration_list = [
								{
									"StartDate": "2025-09-15",
									"EndDate": "2030-09-15",
									"Frequency": "SemiAnnual",
									"Tenor": "ENDDATE",
									"UserTenor": "",
									"Calendar": "Target",
									"ProductType": "Swaption",
									"Strike": "ATM",
									"CalibrationType": "Bootstrap",
									"Parameter": "Volatility"
								}
                        	]
			)]
)
print("	Pricing parameters configured")

Step 1: Configuring instrument definition...
	Instrument definition configured
Step 2: Creating instrument definition object...
	Instrument definition created
Step 3: Configuring pricing parameters...
	Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function with error handling
try:
    # The 'definitions' parameter accepts a list of request items for batch processing
    response = sp.price(
        definitions=[callable_RA],
        pricing_preferences=callable_RA_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
except Exception as e:
    print(f"Price Calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., underlying, payoff description, dates, notional), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method, it is StructuredProductsPricingParameters we used

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` with discount factors
   - **description**: Instrument summary StructuredProductDefinition and also default fields not specified in the StructuredProductDefinition, but used by default in the calculation (e.g. price)
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

The analytics fields are useful for understanding which fields are included by default in the price function, even if they are not explicitly specified in `price` function. We detail below the description, valuation and greeks.

### Description

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Callable_RangeAccrual",
    "dealCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "FixedRateLeg": -42490.069557223,
        "Price": -42490.069557223,
        "PricePercent": -4.249006956,
        "RangeAccrualLeg": 115606.051134755,
        "Reinitialisation": 0.0
    }
}


### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_callable_RA_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_callable_RA_valuation)

,Field,Value
0,marketValueInDealCcy,-42490.069557


#### Greeks

In [6]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,11.461000
1,gammaAmountInDealCcy,-0.005073
2,vegaAmountInDealCcy,0.000000
3,thetaAmountInDealCcy,-2.370570
